# 🎥 Enhanced Video Compressor Bot

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

**🚀 Advanced video compression bot with GPU acceleration and dynamic settings management**

## 📋 Setup Instructions:
1. **Required**: Set your `APP_ID`, `API_HASH`, `BOT_TOKEN`, and `OWNER` in the cell below.
2. **Run the Cell**: Click the play button on the cell below. It will handle the complete setup and then display a live log monitor.
3. **Use Your Bot**: Start chatting with your bot on Telegram. All logs will appear in the cell output.

In [ ]:
#@title 🚀 1. Configure and Run the Bot

#@markdown ### 🔧 Required API Configuration
#@markdown **Fill in your credentials below. All other settings can be changed in the bot via `/settings`.**
APP_ID = ""  #@param {type:"string"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER = "" #@param {type:"string"}

# --- DO NOT EDIT BELOW THIS LINE ---

import os
import subprocess
import threading
import time
import uuid
from IPython.display import display, HTML, Javascript
from datetime import datetime

# --- Global State for Logging ---
log_buffer = []
is_bot_running = threading.Event()
log_container_id = f"log_container_{uuid.uuid4().hex[:8]}"
status_container_id = f"status_container_{uuid.uuid4().hex[:8]}"

def run_shell(command):
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    if process.returncode != 0:
        print(f"❌ Command failed: {command}\nError: {process.stderr}")
        return False
    return True

def setup_bot():
    """Handles the setup process: cloning, installing, and configuring."""
    try:
        print("🚀 Enhanced Video Compressor Bot Setup\n" + "="*60)
        if not all([APP_ID, API_HASH, BOT_TOKEN, OWNER]):
            raise ValueError("API credentials (APP_ID, API_HASH, BOT_TOKEN, OWNER) cannot be empty.")

        print("📥 Cloning repository...")
        if os.path.exists('/content/bot'):
            print("📁 Repository exists, pulling latest changes...")
            os.chdir('/content/bot')
            run_shell("git pull -q")
        else:
            run_shell("git clone -q https://github.com/AdittyaMondal/ColabVideoCompressor.git /content/bot")
            os.chdir('/content/bot')

        print("🐍 Installing Python dependencies...")
        run_shell("pip install -q -r requirements.txt")

        print("⚙️ Creating configuration file...")
        env_content = f"APP_ID='{APP_ID}'\nAPI_HASH='{API_HASH}'\nBOT_TOKEN='{BOT_TOKEN}'\nOWNER='{OWNER}'"
        with open(".env", "w") as f:
            f.write(env_content)
        
        print("\n🎉 Setup complete! Starting bot...")
        print("="*60)
        return True
    except Exception as e:
        print(f"❌ SETUP FAILED: {e}")
        return False

def format_log_line(line):
    """Formats log lines with HTML for better readability."""
    line = line.strip()
    color = "#2f3640"  # Default
    if any(s in line for s in ["ERROR", "FAIL", "❌"]): color = "#ff4757"
    elif any(s in line for s in ["WARNING", "⚠️"]): color = "#ffa502"
    elif any(s in line for s in ["INFO", "Step", "✅"]): color = "#2ed573"
    elif any(s in line for s in ["Starting", "🚀", "🎉"]): color = "#3742fa"
    # Add timestamp and better formatting
    timestamp = datetime.now().strftime('%H:%M:%S')
    return f'<div style="color: {color}; font-family: monospace; white-space: pre-wrap; padding: 2px 5px; margin: 1px 0; border-radius: 3px; font-size: 12px; line-height: 1.4;"><span style="color: #666; font-size: 10px;">[{timestamp}]</span> {line}</div>'

def update_status_display(status_text, status_color):
    """Updates the status display using JavaScript without refresh."""
    js_code = f"""
    var statusElement = document.getElementById('{status_container_id}');
    if (statusElement) {{
        statusElement.innerHTML = `
            <div style="background: #f1f2f6; padding: 15px; border-radius: 10px;">
                <h2 style="margin: 0; color: #192a56;">🤖 Enhanced Video Compressor Bot</h2>
                <p style="margin: 5px 0 0; color: #40739e;">Status: <strong style='color:{status_color};'>{status_text}</strong> | Last Update: {datetime.now().strftime('%H:%M:%S')}</p>
            </div>
        `;
    }}
    """
    display(Javascript(js_code))

def add_log_to_display(log_line):
    """Adds a new log line to the display using JavaScript without refresh."""
    # Escape quotes and newlines for JavaScript
    escaped_log = log_line.replace('\\', '\\\\').replace('`', '\\`').replace('${', '\\${')
    
    js_code = f"""
    var logElement = document.getElementById('{log_container_id}');
    if (logElement) {{
        logElement.innerHTML += `{escaped_log}`;
        
        // Keep only the last 100 log entries
        var logChildren = logElement.children;
        while (logChildren.length > 100) {{
            logElement.removeChild(logChildren[0]);
        }}
        
        // Auto-scroll to the bottom
        logElement.scrollTop = logElement.scrollHeight;
    }}
    """
    display(Javascript(js_code))

def stream_and_log_output(process):
    """Reads stdout from the bot process and updates the display in real-time."""
    for line in iter(process.stdout.readline, ''):
        formatted_line = format_log_line(line)
        log_buffer.append(formatted_line)
        add_log_to_display(formatted_line)
        
        # Update status periodically
        if len(log_buffer) % 10 == 0:  # Every 10 logs
            update_status_display("Running", "#2ed573")
    
    is_bot_running.clear()
    final_log = format_log_line("❌ BOT PROCESS TERMINATED.")
    log_buffer.append(final_log)
    add_log_to_display(final_log)
    update_status_display("Stopped", "#ff4757")

def create_initial_display():
    """Creates the initial HTML display structure."""
    initial_html = f"""
    <div id="{status_container_id}" style="margin-bottom: 10px;">
        <div style="background: #f1f2f6; padding: 15px; border-radius: 10px;">
            <h2 style="margin: 0; color: #192a56;">🤖 Enhanced Video Compressor Bot</h2>
            <p style="margin: 5px 0 0; color: #40739e;">Status: <strong style='color:#2ed573;'>Starting...</strong> | Last Update: {datetime.now().strftime('%H:%M:%S')}</p>
        </div>
    </div>
    
    <div style="background: #ffffff; padding: 15px; border-radius: 8px; border: 1px solid #dfe4ea; font-size: 13px;">
        <h4 style="margin-top: 0; color: #495057;">📜 Live Console Logs</h4>
        <div id="{log_container_id}" style="line-height: 1.4; max-height: 500px; overflow-y: auto; padding: 10px; background: #f8f9fa; border-radius: 5px; border: 1px solid #e9ecef;">
            <div style="color: #6c757d; font-style: italic;">Waiting for bot logs...</div>
        </div>
    </div>
    
    <style>
        #{log_container_id} {{
            font-family: 'Courier New', monospace;
        }}
        #{log_container_id}::-webkit-scrollbar {{
            width: 8px;
        }}
        #{log_container_id}::-webkit-scrollbar-track {{
            background: #f1f1f1;
            border-radius: 10px;
        }}
        #{log_container_id}::-webkit-scrollbar-thumb {{
            background: #c1c1c1;
            border-radius: 10px;
        }}
        #{log_container_id}::-webkit-scrollbar-thumb:hover {{
            background: #a8a8a8;
        }}
    </style>
    """
    display(HTML(initial_html))

def clear_initial_log():
    """Clears the initial 'waiting for logs' message."""
    js_code = f"""
    var logElement = document.getElementById('{log_container_id}');
    if (logElement) {{
        logElement.innerHTML = '';
    }}
    """
    display(Javascript(js_code))

# --- Main Execution Block ---
if setup_bot():
    # Create the initial display
    create_initial_display()
    
    # Start the bot as a background process
    bot_command = "python3 -u -m bot"
    bot_process = subprocess.Popen(
        bot_command.split(),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    # Set the running flag and start the log reader thread
    is_bot_running.set()
    
    # Clear initial message after a short delay
    time.sleep(1)
    clear_initial_log()
    
    # Start the log streaming thread
    log_thread = threading.Thread(target=stream_and_log_output, args=(bot_process,))
    log_thread.daemon = True
    log_thread.start()
    
    # Keep the main thread alive to allow the logging thread to work
    try:
        while is_bot_running.is_set():
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n🛑 Bot interrupted by user")
        bot_process.terminate()
        is_bot_running.clear()
    
    print("\n✅ Bot session completed")
